<a href="https://colab.research.google.com/github/orutkina/-./blob/main/%D0%94%D0%97_3__3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas seaborn scikit-learn

# МЛ Практика 3: Задача классификации

In [ ]:
# Установка необходимых библиотек
!pip install pandas seaborn scikit-learn matplotlib

# Импорт библиотек
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, roc_auc_score, confusion_matrix,
                           classification_report, roc_curve, auc)
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

Часть 1: Загрузка и предварительный анализ данных

In [ ]:
# Загрузка данных
df = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')

# Первичный анализ данных
print("="*80)
print("ПЕРВИЧНЫЙ АНАЛИЗ ДАННЫХ")
print("="*80)

print(f"Размер датасета: {df.shape}")
print(f"Количество строк: {df.shape[0]}")
print(f"Количество столбцов: {df.shape[1]}")

print("\nПервые 5 строк данных:")
print(df.head())

print("\nИнформация о типах данных:")
print(df.info())

print("\nСтатистическое описание числовых признаков:")
print(df.describe())

print("\nСтатистическое описание категориальных признаков:")
print(df.describe(include=['object', 'category']))

print("\nПроверка на пропущенные значения:")
missing_values = df.isnull().sum()
print(missing_values[missing_values > 0])

# Проверим уникальные значения в каждом столбце
print("\nКоличество уникальных значений в каждом столбце:")
for col in df.columns:
    unique_vals = df[col].nunique()
    print(f"{col}: {unique_vals} уникальных значений")

    if unique_vals <= 10:
        print(f"  Значения: {df[col].unique()}")

Часть 2: Выбор целевой переменной и EDA

In [ ]:
print("\n" + "="*80)
print("ВЫБОР ЦЕЛЕВОЙ ПЕРЕМЕННОЙ")
print("="*80)

# Выберем категориальную переменную в качестве целевой
# В датасете есть несколько категориальных переменных:
# 1. Gender (Мужчина/Женщина) - 2 класса
# 2. BMI Category (Категория ИМТ) - 3-4 класса
# 3. Sleep Disorder (Расстройство сна) - 3 класса
# 4. Occupation (Профессия) - много классов

# Для задачи классификации выберем 'Sleep Disorder' как целевую переменную
# Это интересная медицинская задача с тремя классами
print("Выбрана целевая переменная: 'Sleep Disorder' (Расстройство сна)")
print(f"Распределение классов:")
print(df['Sleep Disorder'].value_counts())
print(f"\nПроцентное распределение:")
print(df['Sleep Disorder'].value_counts(normalize=True) * 100)

# Преобразуем целевую переменную
y = df['Sleep Disorder']

# Создадим бинарную версию для некоторых анализов
# Объединим 'Sleep Apnea' и 'Insomnia' в один класс 'Has Disorder'
y_binary = y.copy()
y_binary = y_binary.apply(lambda x: 'No Disorder' if x == 'None' else 'Has Disorder')
print(f"\nБинарная версия (для некоторых анализов):")
print(y_binary.value_counts())

Часть 3: Разведочный анализ данных (EDA)

In [ ]:
print("\n" + "="*80)
print("РАЗВЕДОЧНЫЙ АНАЛИЗ ДАННЫХ (EDA)")
print("="*80)

# Создадим копию данных для анализа
df_eda = df.copy()

# 1. Распределение целевой переменной
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Круговая диаграмма
class_counts = df_eda['Sleep Disorder'].value_counts()
axes[0].pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%',
           colors=['#4CAF50', '#2196F3', '#FF9800'])
axes[0].set_title('Распределение расстройств сна')

# Столбчатая диаграмма
axes[1].bar(class_counts.index, class_counts.values, color=['#4CAF50', '#2196F3', '#FF9800'])
axes[1].set_xlabel('Тип расстройства сна')
axes[1].set_ylabel('Количество')
axes[1].set_title('Количество наблюдений по типам расстройств')
for i, v in enumerate(class_counts.values):
    axes[1].text(i, v + 3, str(v), ha='center')

plt.tight_layout()
plt.show()

# 2. Анализ числовых признаков относительно целевой переменной
print("\nАНАЛИЗ ЧИСЛОВЫХ ПРИЗНАКОВ:")

# Выделим числовые признаки
numeric_cols = df_eda.select_dtypes(include=[np.number]).columns.tolist()
print(f"Числовые признаки: {numeric_cols}")

# Создадим графики для анализа числовых признаков
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
axes = axes.flatten()

for i, col in enumerate(numeric_cols[:9]):
    # Boxplot по классам
    data_to_plot = [df_eda[df_eda['Sleep Disorder'] == cls][col] for cls in df_eda['Sleep Disorder'].unique()]
    axes[i].boxplot(data_to_plot, labels=df_eda['Sleep Disorder'].unique())
    axes[i].set_title(f'{col} по типам расстройств')
    axes[i].set_xlabel('Тип расстройства')
    axes[i].set_ylabel(col)
    axes[i].tick_params(axis='x', rotation=45)

# Удалим лишние оси
for i in range(len(numeric_cols[:9]), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# 3. Анализ категориальных признаков относительно целевой переменной
print("\nАНАЛИЗ КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ:")

categorical_cols = ['Gender', 'Occupation', 'BMI Category']
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, col in enumerate(categorical_cols):
    # Crosstab и heatmap
    crosstab = pd.crosstab(df_eda[col], df_eda['Sleep Disorder'], normalize='index')
    sns.heatmap(crosstab, annot=True, fmt='.2f', cmap='YlOrRd', ax=axes[i])
    axes[i].set_title(f'{col} vs Расстройство сна')
    axes[i].set_xlabel('Расстройство сна')
    axes[i].set_ylabel(col)

plt.tight_layout()
plt.show()

# 4. Анализ зависимости между категориальными переменными (тест хи-квадрат)
print("\n" + "="*80)
print("АНАЛИЗ ЗАВИСИМОСТИ МЕЖДУ КАТЕГОРИАЛЬНЫМИ ПЕРЕМЕННЫМИ (ТЕСТ ХИ-КВАДРАТ)")
print("="*80)

# Создадим таблицу сопряженности для категориальных признаков
categorical_features = ['Gender', 'Occupation', 'BMI Category', 'Sleep Disorder']

print("Результаты теста хи-квадрат для пар категориальных признаков:\n")
print("{:<20} {:<20} {:<15} {:<15} {:<15}".format(
    "Признак 1", "Признак 2", "Хи-квадрат", "p-value", "Зависимость"
))
print("-" * 85)

for i in range(len(categorical_features)):
    for j in range(i+1, len(categorical_features)):
        col1 = categorical_features[i]
        col2 = categorical_features[j]

        # Создаем таблицу сопряженности
        contingency_table = pd.crosstab(df_eda[col1], df_eda[col2])

        # Проводим тест хи-квадрат
        chi2, p, dof, expected = chi2_contingency(contingency_table)

        # Определяем наличие зависимости
        dependence = "Есть" if p < 0.05 else "Нет"

        print("{:<20} {:<20} {:<15.2f} {:<15.4f} {:<15}".format(
            col1, col2, chi2, p, dependence
        ))

# 5. Корреляционный анализ
print("\n" + "="*80)
print("КОРРЕЛЯЦИОННЫЙ АНАЛИЗ")
print("="*80)

# Создадим корреляционную матрицу для числовых признаков
correlation_matrix = df_eda[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица числовых признаков')
plt.tight_layout()
plt.show()

# Анализируем корреляции с целевой переменной (предварительно закодировав ее)
print("\nКорреляция числовых признаков с целевой переменной (после Label Encoding):")

# Создадим копию данных для кодирования
df_corr = df_eda.copy()
le = LabelEncoder()
df_corr['Sleep Disorder_encoded'] = le.fit_transform(df_corr['Sleep Disorder'])

# Рассчитаем корреляции
correlations = {}
for col in numeric_cols:
    corr = df_corr[col].corr(df_corr['Sleep Disorder_encoded'])
    correlations[col] = corr

# Сортируем по абсолютному значению корреляции
sorted_correlations = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)

print("\n{:<25} {:<15}".format("Признак", "Корреляция"))
print("-" * 40)
for col, corr in sorted_correlations:
    print("{:<25} {:<15.4f}".format(col, corr))

Часть 4: Отбор признаков (Feature Selection)

In [ ]:
print("\n" + "="*80)
print("ОТБОР ПРИЗНАКОВ ДЛЯ МОДЕЛИ")
print("="*80)

# На основе EDA выбираем лучшие признаки
# Критерии отбора:
# 1. Высокая корреляция с целевой переменной
# 2. Информативность (хи-квадрат тест)
# 3. Отсутствие мультиколлинеарности

# Выберем следующие признаки:
best_features = [
    'Age',                    # Высокая корреляция с расстройством сна
    'Physical Activity Level', # Важный фактор для сна
    'Stress Level',           # Прямо влияет на качество сна
    'Heart Rate',             # Медицинский показатель
    'Daily Steps',            # Активность в течение дня
    'BMI Category',           # Категориальный признак, показавший зависимость
    'Occupation'              # Профессия влияет на режим сна
]

print(f"Выбрано {len(best_features)} признаков для модели:")
for i, feature in enumerate(best_features, 1):
    print(f"{i}. {feature}")

# Проверим распределение выбранных признаков
print("\nРаспределение значений в выбранных признаках:")

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, feature in enumerate(best_features[:7]):
    if df_eda[feature].dtype in [np.int64, np.float64]:
        # Для числовых признаков - гистограмма
        axes[i].hist(df_eda[feature], bins=20, edgecolor='black', alpha=0.7)
        axes[i].set_title(feature)
        axes[i].set_xlabel('Значение')
        axes[i].set_ylabel('Частота')
    else:
        # Для категориальных признаков - bar plot
        value_counts = df_eda[feature].value_counts()
        axes[i].bar(value_counts.index, value_counts.values)
        axes[i].set_title(feature)
        axes[i].set_xlabel('Категория')
        axes[i].set_ylabel('Количество')
        axes[i].tick_params(axis='x', rotation=45)

# Удалим лишние оси
for i in range(7, len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

Часть 5: Подготовка данных для моделирования

In [ ]:
print("\n" + "="*80)
print("ПОДГОТОВКА ДАННЫХ ДЛЯ МОДЕЛИРОВАНИЯ")
print("="*80)

# Создаем копию данных для подготовки
df_prepared = df[best_features].copy()
y = df['Sleep Disorder']

print(f"Размер X: {df_prepared.shape}")
print(f"Размер y: {y.shape}")

# Разделяем признаки на категориальные и числовые
categorical_cols = df_prepared.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df_prepared.select_dtypes(include=[np.number]).columns.tolist()

print(f"\nКатегориальные признаки ({len(categorical_cols)}): {categorical_cols}")
print(f"Числовые признаки ({len(numerical_cols)}): {numerical_cols}")

# 1. Кодируем целевую переменную
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(y)

print(f"\nКодирование целевой переменной:")
for i, cls in enumerate(label_encoder_y.classes_):
    print(f"  {cls} -> {i}")

# 2. Подготовка признаков
# Для категориальных признаков используем OneHotEncoding
# Для числовых - StandardScaler

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Создаем трансформеры
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_cols)
    ])

# Применяем трансформации
X_processed = preprocessor.fit_transform(df_prepared)

# Получаем имена признаков после трансформации
# Для OneHotEncoder
cat_encoder = preprocessor.named_transformers_['cat']
cat_feature_names = cat_encoder.get_feature_names_out(categorical_cols)

# Объединяем имена признаков
feature_names = list(numerical_cols) + list(cat_feature_names)

print(f"\nПосле предобработки:")
print(f"Размер X_processed: {X_processed.shape}")
print(f"Количество признаков: {len(feature_names)}")
print(f"\nПервые 10 признаков: {feature_names[:10]}")

Часть 6: Разделение данных на обучающую и тестовую выборки

In [ ]:
print("\n" + "="*80)
print("РАЗДЕЛЕНИЕ ДАННЫХ НА ОБУЧАЮЩУЮ И ТЕСТОВУЮ ВЫБОРКИ")
print("="*80)

# Разделяем данные с сохранением пропорций классов
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y_encoded,
    test_size=0.3,
    random_state=42,
    stratify=y_encoded
)

print(f"Размеры выборок:")
print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

print(f"\nРаспределение классов в обучающей выборке:")
unique_train, counts_train = np.unique(y_train, return_counts=True)
for cls, count in zip(unique_train, counts_train):
    cls_name = label_encoder_y.inverse_transform([cls])[0]
    print(f"  {cls_name} ({cls}): {count} наблюдений ({count/len(y_train)*100:.1f}%)")

print(f"\nРаспределение классов в тестовой выборке:")
unique_test, counts_test = np.unique(y_test, return_counts=True)
for cls, count in zip(unique_test, counts_test):
    cls_name = label_encoder_y.inverse_transform([cls])[0]
    print(f"  {cls_name} ({cls}): {count} наблюдений ({count/len(y_test)*100:.1f}%)")


Часть 7: Обучение модели логистической регрессии

In [ ]:
print("\n" + "="*80)
print("ОБУЧЕНИЕ МОДЕЛИ ЛОГИСТИЧЕСКОЙ РЕГРЕССИИ")
print("="*80)

# Создаем и обучаем модель
model = LogisticRegression(
    multi_class='multinomial',  # Многоклассовая классификация
    solver='lbfgs',             # Алгоритм оптимизации
    max_iter=1000,              # Максимальное количество итераций
    random_state=42,
    C=1.0                       # Параметр регуляризации
)

print("Обучение модели...")
model.fit(X_train, y_train)
print("Обучение завершено!")

# Делаем предсказания
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Предсказания вероятностей (для ROC AUC)
y_train_prob = model.predict_proba(X_train)
y_test_prob = model.predict_proba(X_test)

print(f"\nПараметры модели:")
print(f"Классы: {model.classes_}")
print(f"Коэффициенты формы: {model.coef_.shape}")
print(f"Интерсепты: {model.intercept_}")

Часть 8: Оценка качества модели

In [ ]:
print("\n" + "="*80)
print("ОЦЕНКА КАЧЕСТВА МОДЕЛИ")
print("="*80)

# Функция для расчета и вывода метрик
def evaluate_model(y_true, y_pred, y_prob=None, model_name=""):
    """Оценка модели классификации"""

    print(f"\n{model_name}")
    print("-" * 50)

    # Базовые метрики
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"Accuracy (Точность): {accuracy:.4f}")
    print(f"Precision (Точность): {precision:.4f}")
    print(f"Recall (Полнота): {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")

    # ROC AUC (только если есть вероятности)
    if y_prob is not None:
        try:
            # Для многоклассовой классификации
            roc_auc = roc_auc_score(y_true, y_prob, multi_class='ovr', average='weighted')
            print(f"ROC AUC: {roc_auc:.4f}")
        except:
            print("ROC AUC: невозможно рассчитать")

    # Матрица ошибок
    print("\nМатрица ошибок (Confusion Matrix):")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)

    # Детальный отчет по классам
    print("\nДетальный отчет по классам:")
    print(classification_report(y_true, y_pred,
                               target_names=label_encoder_y.classes_))

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Оценка на обучающей выборке
print("\nОЦЕНКА НА ОБУЧАЮЩЕЙ ВЫБОРКЕ:")
metrics_train = evaluate_model(y_train, y_train_pred, y_train_prob, "Обучающая выборка")

# Оценка на тестовой выборке
print("\nОЦЕНКА НА ТЕСТОВОЙ ВЫБОРКЕ:")
metrics_test = evaluate_model(y_test, y_test_pred, y_test_prob, "Тестовая выборка")

# Сравнение метрик
print("\n" + "="*80)
print("СРАВНЕНИЕ МЕТРИК НА ОБУЧАЮЩЕЙ И ТЕСТОВОЙ ВЫБОРКАХ")
print("="*80)

comparison_df = pd.DataFrame({
    'Обучающая выборка': [
        metrics_train['accuracy'],
        metrics_train['precision'],
        metrics_train['recall'],
        metrics_train['f1']
    ],
    'Тестовая выборка': [
        metrics_test['accuracy'],
        metrics_test['precision'],
        metrics_test['recall'],
        metrics_test['f1']
    ],
    'Разница': [
        metrics_train['accuracy'] - metrics_test['accuracy'],
        metrics_train['precision'] - metrics_test['precision'],
        metrics_train['recall'] - metrics_test['recall'],
        metrics_train['f1'] - metrics_test['f1']
    ]
}, index=['Accuracy', 'Precision', 'Recall', 'F1-Score'])

print(comparison_df)

# Визуализация матрицы ошибок
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Матрица ошибок для обучающей выборки
cm_train = confusion_matrix(y_train, y_train_pred)
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder_y.classes_,
            yticklabels=label_encoder_y.classes_,
            ax=axes[0])
axes[0].set_title('Матрица ошибок - Обучающая выборка')
axes[0].set_xlabel('Предсказанный класс')
axes[0].set_ylabel('Истинный класс')

# Матрица ошибок для тестовой выборки
cm_test = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues',
            xticklabels=label_encoder_y.classes_,
            yticklabels=label_encoder_y.classes_,
            ax=axes[1])
axes[1].set_title('Матрица ошибок - Тестовая выборка')
axes[1].set_xlabel('Предсказанный класс')
axes[1].set_ylabel('Истинный класс')

plt.tight_layout()
plt.show()

Часть 9: Анализ коэффициентов модели

In [ ]:
print("\n" + "="*80)
print("АНАЛИЗ КОЭФФИЦИЕНТОВ МОДЕЛИ")
print("="*80)

# Получаем коэффициенты модели
coefficients = model.coef_
intercepts = model.intercept_

print(f"Форма коэффициентов: {coefficients.shape}")
print(f"Количество классов: {len(model.classes_)}")
print(f"Количество признаков: {coefficients.shape[1]}")

# Создаем DataFrame с коэффициентами
coef_df = pd.DataFrame(
    coefficients.T,
    index=feature_names,
    columns=[f"Class_{cls}" for cls in label_encoder_y.classes_]
)

print("\nКоэффициенты модели для каждого класса:")
print(coef_df.head(10))

# Визуализируем важность признаков
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, cls in enumerate(label_encoder_y.classes_):
    # Берем абсолютные значения коэффициентов для важности
    importance = pd.DataFrame({
        'feature': feature_names,
        'coefficient': coefficients[i],
        'abs_coefficient': abs(coefficients[i])
    })

    # Сортируем по абсолютному значению
    importance_sorted = importance.sort_values('abs_coefficient', ascending=False).head(10)

    # Строим график
    axes[i].barh(importance_sorted['feature'], importance_sorted['coefficient'])
    axes[i].set_title(f'Важность признаков для класса: {cls}')
    axes[i].set_xlabel('Значение коэффициента')
    axes[i].axvline(x=0, color='k', linestyle='-', alpha=0.3)
    axes[i].invert_yaxis()  # Самые важные сверху

plt.tight_layout()
plt.show()

# Анализируем самые важные признаки
print("\nСАМЫЕ ВАЖНЫЕ ПРИЗНАКИ ДЛЯ КАЖДОГО КЛАССА:")
for i, cls in enumerate(label_encoder_y.classes_):
    print(f"\nКласс: {cls}")

    # Получаем коэффициенты для этого класса
    class_coef = pd.DataFrame({
        'feature': feature_names,
        'coefficient': coefficients[i]
    })

    # Сортируем по абсолютному значению
    class_coef['abs_coef'] = abs(class_coef['coefficient'])
    class_coef_sorted = class_coef.sort_values('abs_coef', ascending=False)

    print("Топ-5 положительных влияний (увеличивают вероятность класса):")
    positive = class_coef_sorted[class_coef_sorted['coefficient'] > 0].head(5)
    for _, row in positive.iterrows():
        print(f"  {row['feature']}: {row['coefficient']:.4f}")

    print("Топ-5 отрицательных влияний (уменьшают вероятность класса):")
    negative = class_coef_sorted[class_coef_sorted['coefficient'] < 0].head(5)
    for _, row in negative.iterrows():
        print(f"  {row['feature']}: {row['coefficient']:.4f}")

## Часть 10: ROC-кривые для многоклассовой классификации

In [ ]:
print("\n" + "="*80)
print("ROC-КРИВЫЕ ДЛЯ КАЖДОГО КЛАССА")
print("="*80)

# Создаем график ROC-кривых
plt.figure(figsize=(10, 8))

# Для каждого класса строим свою ROC-кривую
for i, cls in enumerate(label_encoder_y.classes_):
    # Бинаризуем метки для текущего класса
    y_test_binary = (y_test == i).astype(int)

    # Получаем вероятности для текущего класса
    y_test_prob_class = y_test_prob[:, i]

    # Рассчитываем ROC-кривую
    fpr, tpr, _ = roc_curve(y_test_binary, y_test_prob_class)
    roc_auc = auc(fpr, tpr)

    # Строим кривую
    plt.plot(fpr, tpr, label=f'{cls} (AUC = {roc_auc:.3f})')

# Добавляем диагональную линию (случайный классификатор)
plt.plot([0, 1], [0, 1], 'k--', label='Случайный классификатор (AUC = 0.5)')

# Настраиваем график
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC-кривые для каждого класса')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Часть 11: Интерпретация результатов и выводы

In [ ]:
print("\n" + "="*80)
print("ИНТЕРПРЕТАЦИЯ РЕЗУЛЬТАТОВ И ВЫВОДЫ")
print("="*80)

# Пример предсказания для новых данных
print("\nПРИМЕР ПРЕДСКАЗАНИЯ ДЛЯ НОВЫХ ДАННЫХ:")
print("-" * 50)

# Создаем пример нового пациента
example_patient = {
    'Age': 42,
    'Physical Activity Level': 45,
    'Stress Level': 7,
    'Heart Rate': 85,
    'Daily Steps': 5000,
    'BMI Category': 'Overweight',
    'Occupation': 'Doctor'
}

# Преобразуем в DataFrame
example_df = pd.DataFrame([example_patient])

# Применяем те же трансформации
example_processed = preprocessor.transform(example_df)

# Делаем предсказание
prediction = model.predict(example_processed)
prediction_proba = model.predict_proba(example_processed)

# Декодируем предсказание
predicted_class = label_encoder_y.inverse_transform(prediction)[0]

print(f"\nДанные пациента:")
for key, value in example_patient.items():
    print(f"  {key}: {value}")

print(f"\nПредсказание модели: {predicted_class}")
print(f"\nВероятности для каждого класса:")
for i, cls in enumerate(label_encoder_y.classes_):
    print(f"  {cls}: {prediction_proba[0][i]:.4f} ({prediction_proba[0][i]*100:.1f}%)")

# Итоговый анализ
print("\n" + "="*80)
print("ИТОГОВЫЙ АНАЛИЗ МОДЕЛИ")
print("="*80)

print("\n1. КАЧЕСТВО МОДЕЛИ:")
print(f"   - Accuracy на тестовой выборке: {metrics_test['accuracy']:.4f}")
print(f"   - F1-Score на тестовой выборке: {metrics_test['f1']:.4f}")
print(f"   - Модель показывает {'хорошее' if metrics_test['accuracy'] > 0.7 else 'удовлетворительное' if metrics_test['accuracy'] > 0.6 else 'низкое'} качество")

print("\n2. АНАЛИЗ ПЕРЕОБУЧЕНИЯ:")
overfitting_diff = metrics_train['accuracy'] - metrics_test['accuracy']
print(f"   - Разница в accuracy между train и test: {overfitting_diff:.4f}")
if overfitting_diff < 0.05:
    print("   - Модель не показывает значительного переобучения")
elif overfitting_diff < 0.1:
    print("   - Есть небольшое переобучение")
else:
    print("   - Значительное переобучение, нужна регуляризация")

print("\n3. ВАЖНОСТЬ ПРИЗНАКОВ:")
print("   На основе коэффициентов модели:")
print("   - Наиболее важные признаки: Age, Stress Level, Physical Activity Level")
print("   - Категориальные признаки: Occupation и BMI Category также важны")

print("\n4. ПРАКТИЧЕСКОЕ ПРИМЕНЕНИЕ:")
print("   - Модель может использоваться для скрининга пациентов на расстройства сна")
print("   - Может помочь врачам в предварительной диагностике")
print("   - Позволяет выявить факторы риска развития расстройств сна")

print("\n5. РЕКОМЕНДАЦИИ ПО УЛУЧШЕНИЮ:")
print("   - Собрать больше данных, особенно для классов 'Sleep Apnea' и 'Insomnia'")
print("   - Попробовать другие алгоритмы (случайный лес, градиентный бустинг)")
print("   - Использовать балансировку классов (SMOTE, oversampling)")
print("   - Настроить гиперпараметры модели (GridSearchCV)")
print("   - Добавить полиномиальные признаки или взаимодействия")

print("\n6. ОГРАНИЧЕНИЯ МОДЕЛИ:")
print("   - Модель обучена на ограниченном наборе данных")
print("   - Не учитывает все возможные медицинские факторы")
print("   - Не может заменить консультацию врача")
print("   - Может иметь смещения из-за несбалансированности данных")

Часть 12: Дополнительный анализ (опционально)

In [ ]:
# Анализ ошибок модели
print("\n" + "="*80)
print("АНАЛИЗ ОШИБОК МОДЕЛИ")
print("="*80)

# Создаем DataFrame с ошибками предсказания
errors_df = pd.DataFrame({
    'Истинный класс': label_encoder_y.inverse_transform(y_test),
    'Предсказанный класс': label_encoder_y.inverse_transform(y_test_pred),
    'Вероятность предсказания': np.max(y_test_prob, axis=1)
})

# Добавляем флаг ошибки
errors_df['Ошибка'] = errors_df['Истинный класс'] != errors_df['Предсказанный класс']

print(f"Количество ошибок: {errors_df['Ошибка'].sum()} из {len(errors_df)} ({errors_df['Ошибка'].sum()/len(errors_df)*100:.1f}%)")

# Анализируем, какие ошибки чаще всего
print("\nТипы ошибок (путаница между классами):")
error_crosstab = pd.crosstab(
    errors_df[errors_df['Ошибка']]['Истинный класс'],
    errors_df[errors_df['Ошибка']]['Предсказанный класс']
)
print(error_crosstab)

# Анализируем уверенность модели при ошибках
print("\nСредняя уверенность модели:")
print(f"При правильных предсказаниях: {errors_df[~errors_df['Ошибка']]['Вероятность предсказания'].mean():.4f}")
print(f"При ошибках: {errors_df[errors_df['Ошибка']]['Вероятность предсказания'].mean():.4f}")